In [10]:
import torch
import sys
sys.path.append('/Users/david/Desktop/CSE60657/Project/NLP_project/')
#import layers
#import seqlabel
from SpanBERT import*
import utils
model = torch.load("./best_model.pt", map_location=torch.device('cpu'))

In [11]:
annotation_dict = utils.load_data("./Data/qed-dev.jsonlines")
M = annotation_dict[-4340755100872459608]
text = M.passage
question = M.question
coreference = M.aligned_nps
answer_ls = M.answer
selected_sent = M.selected_sent
sentence_starts = []
if len(M.sentence_starts) == 0:
    sentence_starts = [0, selected_sent['end']]
else: 
    sentence_starts = M.sentence_starts

true_sent_idx = utils.get_true_sent_idx(selected_sent, sentence_starts)
sents_ls = utils.separate_sents(text, sentence_starts)

encoded_text = model.tokenize(text)
encoded_question = model.tokenize(question)

question_span = utils.get_encoded_span(question, coreference, type=0)
encoded_question_span = model.tokenize(question_span)
question_start, question_end = find_start_end_pos(encoded_question, encoded_question_span)

text_span = utils.get_encoded_span(text, coreference, type=1)
encoded_text_span = model.tokenize(text_span)
encoded_text = model.tokenize(text)
text_start, text_end = find_start_end_pos(encoded_text, encoded_text_span)

answer_span = utils.get_answer_span(text, answer_ls)
encoded_answer_span = model.tokenize(answer_span)
answer_start, answer_end = find_start_end_pos(encoded_text, encoded_answer_span)

encoded_sents_ls = []
for sents in sents_ls:
    encoded_sent = model.tokenize(sents)
    encoded_sents_ls.append(encoded_sent)


In [18]:
M.question

'what does hp mean in war and order'

In [ ]:
M.selected_sent

In [20]:
M.passage

'Health or vitality is an attribute assigned to entities , such as the player character , enemies and objects within a role - playing or video game , that indicates its state in combat . Health is usually measured in hit points or health points , shortened to HP . When the HP of a player character reaches zero , the player may lose a life or their character might become incapacitated or die . When the HP of an enemy reaches zero , it may be defeated or die and the player is usually rewarded in some way .'

In [21]:
M.answer

[Entity(start_offset=216, end_offset=243, type='context', text='hit points or health points', normalized_text='hit points or health points')]

In [12]:
M.aligned_nps

[(Entity(start_offset=10, end_offset=12, type='question', text='hp', normalized_text='hp'),
  Entity(start_offset=259, end_offset=261, type='context', text='HP', normalized_text='hp'))]

In [13]:
M.passage

'Health or vitality is an attribute assigned to entities , such as the player character , enemies and objects within a role - playing or video game , that indicates its state in combat . Health is usually measured in hit points or health points , shortened to HP . When the HP of a player character reaches zero , the player may lose a life or their character might become incapacitated or die . When the HP of an enemy reaches zero , it may be defeated or die and the player is usually rewarded in some way .'

In [14]:
coreference_span, coreference_pos = model.find_most_similar_span(encoded_text, encoded_question, question_start, question_end, window_size=2)
f1_score, answer_span, answer_pos = model.find_answer(encoded_text, encoded_question, answer_start, answer_end)
val = model.find_sentence(encoded_question, encoded_sents_ls, true_sent_idx)

In [15]:
coreference_span

'hp of a'

In [16]:
coreference_pos

(53, 56)

In [17]:
answer_span

'might become incapacitated or die. when the hp of an enemy reaches zero, it may be defeated or die and the player is usually rewarded'

In [19]:
def find_sentence(model, encoded_question, encoded_sents_ls, true_sent_idx):
    question_embeddings = model.model(**encoded_question).last_hidden_state
    sents_embedding_ls = []
    for sents in encoded_sents_ls:
        sent_embedding = model.model(**sents).last_hidden_state
        sents_embedding_ls.append(sent_embedding)
    _, similarity_scores = model.sents_cos_similarity(question_embeddings, sents_embedding_ls, true_sent_idx)
    pred_idx = similarity_scores.argmax().item()
    return (pred_idx, true_sent_idx)

selected_sent = M.selected_sent
sentence_starts = []
if len(M.sentence_starts) == 0:
    sentence_starts = [0, selected_sent['end']]
else: 
    sentence_starts = M.sentence_starts
true_sent_idx = utils.get_true_sent_idx(selected_sent, sentence_starts)

find_sentence(model, encoded_question, encoded_sents_ls, true_sent_idx)

(1, 1)